In [3]:
from PyPDF2 import PdfReader
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

In [4]:
# Load NER Model
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [5]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text
# Function to run NER and return DataFrame
def get_entities(text):
    ner_results = ner_pipeline(text)
    df = pd.DataFrame(ner_results)
    # Optional: drop duplicates
    df = df.drop_duplicates(subset=["word", "entity"])
    return df    

In [7]:
pdf_path = "data/sample.pdf"  # Replace with your PDF
text = extract_text_from_pdf(pdf_path)
df_entities = get_entities(text)
print(df_entities.head())

  entity     score  index   word  start  end
0  B-PER  0.995973      5   John     14   18
1  I-PER  0.997951      6     Do     19   21
2  I-PER  0.842430      7    ##e     21   22
3  B-PER  0.672285     47  Smith    154  159
4  B-PER  0.981908     52  Alice    176  181


In [9]:
# Save results to CSV
df_entities.to_csv("ner_results.csv", index=False)